In [ ]:
import torch
import torchvision
import numpy as np
import logging
import random
from scipy import spatial
from utils import Utils
import os

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(name)-12s %(levelname)-8s %(message)s")

from benchmark import ImageBenchmark
bench = ImageBenchmark()
models = list(bench.list_models())
models_dict = {}
for i, model in enumerate(models):
    if not model.torch_model_exists():
        continue
    print(f'{i}\t {model.__str__()}')
    models_dict[model.__str__()] = model

In [ ]:
DEVICE = 'cuda'

def gen_adv_inputs(model, inputs):
    from advertorch.attacks import LinfPGDAttack
    def myloss(yhat, y):
        return -((yhat[:,0]-y[:,0])**2 + 0.1*((yhat[:,1:]-y[:,1:])**2).mean(1)).mean()
        
    model = model.to(DEVICE)
    inputs = torch.from_numpy(inputs).to(DEVICE)
    with torch.no_grad():
        model.eval()
        clean_outputs = model(inputs)
    
    output_shape = clean_outputs.shape
    batch_size = output_shape[0]
    num_classes = output_shape[1]
    
    output_mean = clean_outputs.mean(axis=0)
    target_outputs = output_mean - clean_outputs
    
    y = torch.zeros(size=output_shape).to(DEVICE)
    y[:, :] = 100000
    # more diversity
    y = target_outputs * 1000
#     rand_idx = torch.randint(low=0, high=num_classes, size=(batch_size,))
#     y = torch.nn.functional.one_hot(rand_idx, num_classes=num_classes).to(DEVICE) * 10
#     print(y)
    
    adversary = LinfPGDAttack(
        model, loss_fn=myloss, eps=0.1,
        nb_iter=50, eps_iter=0.01, 
        rand_init=True, clip_min=inputs.min().item(), clip_max=inputs.max().item(),
        targeted=True
    )
    
    adv_inputs = adversary.perturb(inputs, y)
    
    with torch.no_grad():
        model.eval()
        adv_outputs = model(adv_inputs).to('cpu').numpy()
#     print(adv_outputs)
    torch.cuda.empty_cache()
    return adv_inputs.to('cpu').numpy()


model = models_dict['pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-prune(0.2)-']
model.torch_model.to(DEVICE)
seed_inputs = model.get_seed_inputs(100, rand=False)
seed_outputs = model.batch_forward(seed_inputs)
_, seed_preds = seed_outputs.to('cpu').data.max(1)

from datetime import datetime
start_time = datetime.now()
adv_inputs = gen_adv_inputs(model.torch_model, seed_inputs)
adv_outputs = model.batch_forward(adv_inputs)
_, adv_preds = adv_outputs.to('cpu').data.max(1)
model.torch_model.cpu()

time_spent = (datetime.now() - start_time).total_seconds()
print(f'spent {time_spent} seconds')

print(f"seed_preds={seed_preds}, adv_preds={adv_preds}, seed_outputs={seed_outputs}, adv_outputs={adv_outputs}")

In [ ]:
DEVICE = 'cuda'


class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    # Background colors:
    GREYBG = '\033[100m'
    REDBG = '\033[101m'
    GREENBG = '\033[102m'
    YELLOWBG = '\033[103m'
    BLUEBG = '\033[104m'
    PINKBG = '\033[105m'
    CYANBG = '\033[106m'


def gen_adv_inputs(model, inputs):
    from advertorch.attacks import LinfPGDAttack
    def myloss(yhat, y):
        return -((yhat[:,0]-y[:,0])**2 + 0.1*((yhat[:,1:]-y[:,1:])**2).mean(1)).mean()
        
    model = model.to(DEVICE)
    inputs = torch.from_numpy(inputs).to(DEVICE)
    with torch.no_grad():
        model.eval()
        clean_outputs = model(inputs)
    
    output_shape = clean_outputs.shape
    batch_size = output_shape[0]
    num_classes = output_shape[1]
    
    output_mean = clean_outputs.mean(axis=0)
    target_outputs = output_mean - clean_outputs
    
#     # No diversity, high divergence
#     y = torch.zeros(size=output_shape).to(DEVICE)
#     y[:, 0] = 100000  # Low diversity, high divergence
#     y[:] = output_mean  # Low diversity, low divergence
    
#     y = target_outputs
#     y = target_outputs * 0.1  # High diversity, low divergence 
    y = target_outputs * 1000  # High diversity, high divergence
    
    adversary = LinfPGDAttack(
        model, loss_fn=myloss, eps=0.06,
        nb_iter=50, eps_iter=0.01, 
        rand_init=True, clip_min=inputs.min().item(), clip_max=inputs.max().item(),
        targeted=True
    )
    
    adv_inputs = adversary.perturb(inputs, y)
    
    with torch.no_grad():
        model.eval()
        adv_outputs = model(adv_inputs).to('cpu').numpy()
#     print(adv_outputs)
    torch.cuda.empty_cache()
    return adv_inputs.to('cpu').numpy()


def get_comparable_models(target_model):
    target_model_name = target_model.__str__()
    target_model_segs = target_model_name.split('-')
    parent_model_name = '-'.join(target_model_segs[:-2]) + '-'
    parent_model = models_dict[parent_model_name]
    # print(f'parent_model: {parent_model}')
    reference_models = []
    for model in models:
        if not model.__str__().startswith(target_model_segs[0]):
            reference_models.append(model)
            # print(f'reference_model: {model}')
    return parent_model, reference_models


def compute_ddv(model, normal_inputs, adv_inputs):
    normal_outputs = model.batch_forward(normal_inputs).cpu().numpy()
    adv_outputs = model.batch_forward(adv_inputs).cpu().numpy()
    output_pairs = zip(normal_outputs, adv_outputs)
    # print(list(output_pairs)[0])
    ddv = []  # DDV is short for decision distance vector
    for i, (ya, yb) in enumerate(output_pairs):
        dist = spatial.distance.cosine(ya, yb)
        ddv.append(dist)
    ddv = Utils.normalize(np.array(ddv))
    return ddv


def load_inputs(model):
    inputs_path = os.path.join(model.torch_model_path, 'inputs.npz')
    npzfile = np.load(inputs_path, allow_pickle=True)
    seed_inputs = npzfile['seed_inputs']
    adv_inputs = npzfile['adv_inputs']
    saved_inputs = npzfile['saved_inputs'].item()
    # print(saved_inputs)
    return seed_inputs, adv_inputs
    

skip_steal_homo = True
skip_quant_float = True

for i, model in enumerate(models):
    if not model.torch_model_exists():
        continue
    model_name = model.__str__()
    if not model_name.startswith('pretrain'):
        continue
    if len(model_name.split('-')) < 3:
        continue
    if i < 6:
        continue
    if 'quantize(float16)' in model_name and skip_quant_float:
        continue
    if 'steal' in model_name and skip_steal_homo:
        arch1 = model_name[model_name.find('(')+1:model_name.find(',')]
        arch2 = model_name[model_name.rfind('(')+1:model_name.rfind(')')]
        if arch1 == arch2:
            continue

#     if 'steal' not in model_name:
#         continue        
#     if 'quantize' not in model_name:
#         continue
#     if not model_name.endswith('prune(0.8)-'):
#         continue
#     if i != 50:
#         continue
    
    parent_model, ref_models = get_comparable_models(model)
    print(f'{i}\t {model_name} testing')
    
    source_model = model
    if 'quantize' in model_name:
        model.torch_model.cpu()
        source_model = parent_model
    
    seed_inputs = model.get_seed_inputs(100, rand=False)
#     seed_inputs = np.array([seed_inputs[0]] * 100)  # remove diversity of inputs
    adv_inputs = gen_adv_inputs(source_model.torch_model, seed_inputs)
    
#     adv_inputs = model.get_seed_inputs(100, rand=False)  # all normal inputs

    # all adversarial inputs
#     adv_inputs2 = gen_adv_inputs(source_model.torch_model, seed_inputs)
#     seed_inputs = adv_inputs2

#     seed_inputs, adv_inputs = load_inputs(model)  # load saved inputs
    source_model.torch_model.to(DEVICE)
    ddv = compute_ddv(model, seed_inputs, adv_inputs)
    # print(f'self_sim: {spatial.distance.cosine(ddv, ddv):.4f}')
    
    parent_sim = 0
    gap_min = 100
    for i, ref_model in enumerate([parent_model] + ref_models):
        if 'quantize' in ref_model.__str__(): # quantized models are equivalent to its teacher model
            continue
        try:
            ref_model.torch_model.to(DEVICE)
            ref_ddv = compute_ddv(ref_model, seed_inputs, adv_inputs)
            ref_sim = spatial.distance.cosine(ddv, ref_ddv)
            ref_model.torch_model.cpu()
            if i == 0:
                parent_sim = ref_sim
                gap = 1
                print(f'parent_sim: {ref_sim:.4f} {ref_model}')
            else:
                gap = ref_sim - parent_sim
                if gap > 0:
                    print(f'ref_sim: {ref_sim:.4f} gap={gap:.4f} {ref_model}')
                else:
                    print(f'{bcolors.WARNING}[ERROR] ref_sim: {ref_sim:.4f} gap={gap:.4f} {ref_model}{bcolors.ENDC}')
                if gap < gap_min:
                    gap_min = gap
        except Exception as e:
            print(f'failed to compare: {ref_model}')
            print(f'exception: {e}')
    print(f'--gap_min:{gap_min:.4f}, parent_sim:{parent_sim:.4f}, correct:{gap_min>0}, model:{model_name}')
    # break
    source_model.torch_model.cpu()

In [ ]:
x = np.random.randn(10, 3, 3)
y = np.array([x[0]] * 10)
print(x, y)